# KickStarter Project Analysis

In [72]:
import pyspark
from pyspark.sql import SparkSession
from itertools import islice
from datetime import date
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DateType
from pyspark.sql.types import StructField 
from pyspark.sql.types import StructType
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date
from pyspark.sql import Row
from pyspark.sql.functions import max as max_
from pyspark.sql.functions import sum as sum_
from pyspark.sql.functions import desc
from pyspark.sql.functions import lit
from pyspark.sql.functions import when
from pyspark.sql.functions import round as round_
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank
from pyspark.sql.functions import countDistinct
import math

In [2]:
sc = pyspark.SparkContext()

In [3]:
spark = SparkSession(sc)

In [4]:
# source: https://www.kaggle.com/kemical/kickstarter-projects/data#
ks = sc.textFile('input/ks-projects-2018.csv')

In [5]:
col_names = ks.take(1)

In [6]:
col_names = [c for c in col_names[0].split(",")]

In [7]:
print(col_names)
print(len(col_names))

['ID', 'name', 'category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged', 'usd_pledged_real', 'usd_goal_real']
15


In [8]:
# skip the first row
ks = ks.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it)

In [9]:
# filter out row with more than 15 columns
parts = ks.map(lambda l: l.split(","))\
          .filter(lambda row: len(row) <= 15)\
          .map(lambda p: tuple(i for i in p))

In [10]:
parts.take(2)

[('1000002330',
  'The Songs of Adelaide & Abullah',
  'Poetry',
  'Publishing',
  'GBP',
  '2015-10-09',
  '1000.00',
  '2015-08-11 12:12:28',
  '0.00',
  'failed',
  '0',
  'GB',
  '0.00',
  '0.00',
  '1533.95'),
 ('1000003930',
  'Greeting From Earth: ZGAC Arts Capsule For ET',
  'Narrative Film',
  'Film & Video',
  'USD',
  '2017-11-01',
  '30000.00',
  '2017-09-02 04:43:57',
  '2421.00',
  'failed',
  '15',
  'US',
  '100.00',
  '2421.00',
  '30000.00')]

### Specifying schema

In [11]:
fields = [StructField(name, StringType(), True) for name in col_names]

In [12]:
schema = StructType(fields)

In [13]:
projects_df = spark.createDataFrame(parts, schema)

In [14]:
projects_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- goal: string (nullable = true)
 |-- launched: string (nullable = true)
 |-- pledged: string (nullable = true)
 |-- state: string (nullable = true)
 |-- backers: string (nullable = true)
 |-- country: string (nullable = true)
 |-- usd pledged: string (nullable = true)
 |-- usd_pledged_real: string (nullable = true)
 |-- usd_goal_real: string (nullable = true)



In [15]:
projects_df = projects_df.withColumn("deadline_date", col("deadline").cast(DateType()))

In [16]:
projects_df = projects_df.withColumn("launched_date", col("launched").cast(DateType()))

In [17]:
projects_df = projects_df.withColumn("goal_int", col("goal").cast(DoubleType()))
projects_df = projects_df.withColumn("pledged_int", col("pledged").cast(DoubleType()))
projects_df = projects_df.withColumn("backers_int", col("backers").cast(IntegerType()))
projects_df = projects_df.withColumn("usd_pledged_int", col("usd pledged").cast(DoubleType()))
projects_df = projects_df.withColumn("usd_pledged_real_int", col("usd_pledged_real").cast(DoubleType()))
projects_df = projects_df.withColumn("usd_goal_real_int", col("usd_goal_real").cast(DoubleType()))

In [18]:
projects_df = projects_df.drop("deadline", "launched", "goal", "pledged", "backers", "usd pledged", "usd_pledged_real", "usd_goal_real")

In [19]:
projects_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- deadline_date: date (nullable = true)
 |-- launched_date: date (nullable = true)
 |-- goal_int: double (nullable = true)
 |-- pledged_int: double (nullable = true)
 |-- backers_int: integer (nullable = true)
 |-- usd_pledged_int: double (nullable = true)
 |-- usd_pledged_real_int: double (nullable = true)
 |-- usd_goal_real_int: double (nullable = true)



## Initial Exploration

In [87]:
projects_df = projects_df.repartition(2)

In [88]:
projects_df.rdd.getNumPartitions()

2

In [82]:
# persist the dataframe cause we need query against this table several times
projects_df.unpersist()

DataFrame[ID: string, name: string, category: string, main_category: string, currency: string, state: string, country: string, deadline_date: date, launched_date: date, goal_int: double, pledged_int: double, backers_int: int, usd_pledged_int: double, usd_pledged_real_int: double, usd_goal_real_int: double]

#### Latest Launched Date

In [20]:
projects_df.groupBy().agg(max_('launched_date')).collect()

[Row(max(launched_date)=datetime.date(2018, 1, 2))]

#### Distribution of Project State

In [21]:
projects_df.groupBy('state').count().sort(desc("count")).show()

+----------+------+
|     state| count|
+----------+------+
|    failed|185167|
|successful|124482|
|  canceled| 36272|
|      live|  2619|
| suspended|  1723|
+----------+------+



#### Top Countries that Launched Projects

In [23]:
projects_df.groupBy('country').count().sort(desc("count")).show(10)

+-------+------+
|country| count|
+-------+------+
|     US|273134|
|     GB| 31799|
|     CA| 13881|
|     AU|  7367|
|     DE|  3925|
|     IT|  2639|
|     FR|  2632|
|     NL|  2615|
|     ES|  2113|
|     SE|  1674|
+-------+------+
only showing top 10 rows



#### Success Rate for Countires

In [31]:
projects_df.filter((col('state') == 'failed') | (col('state') == 'successful')) \
           .withColumn('dummy', lit(1)) \
           .withColumn('succ_boolean', when(col('state') == 'successful',1).otherwise(0))\
           .groupBy('country').agg(sum_('succ_boolean').alias('succ'), sum_('dummy').alias('ttl')) \
           .withColumn('percentage', round_(col('succ')/col('ttl'),2))\
           .sort(desc('percentage'))\
           .show()

+-------+------+------+----------+
|country|  succ|   ttl|percentage|
+-------+------+------+----------+
|     HK|   194|   439|      0.44|
|     US|101475|243856|      0.42|
|     GB| 11400| 27826|      0.41|
|     SG|   168|   430|      0.39|
|     DK|   345|   880|      0.39|
|     NZ|   427|  1199|      0.36|
|     FR|   808|  2261|      0.36|
|     SE|   484|  1436|      0.34|
|     LU|    18|    54|      0.33|
|     CA|  3880| 11641|      0.33|
|     JP|     7|    22|      0.32|
|     AU|  1882|  6222|       0.3|
|     IE|   181|   629|      0.29|
|     BE|   140|   485|      0.29|
|     CH|   171|   600|      0.29|
|     MX|   372|  1316|      0.28|
|     DE|   874|  3243|      0.27|
|     NO|   147|   542|      0.27|
|     ES|   459|  1744|      0.26|
|     NL|   552|  2201|      0.25|
+-------+------+------+----------+
only showing top 20 rows



#### Top Successful Main Categories

In [63]:
projects_df.filter(col('state') == 'successful')\
           .groupBy('main_category') \
           .count().sort(desc('count')) \
           .select('main_category','count').show(10)

+-------------+-----+
|main_category|count|
+-------------+-----+
|        Music|22588|
| Film & Video|22436|
|        Games|11771|
|   Publishing|11242|
|          Art|10724|
|       Design| 9592|
|      Theater| 6018|
|   Technology| 5814|
|       Comics| 5521|
|         Food| 5496|
+-------------+-----+
only showing top 10 rows



#### Success Rate for Each Sub Category within the Main Category

In [69]:
#.withColumn('percentage', col('sum(succ_boolean)')/col('sum(dummy)').over(Window.partitionBy(projects_df['country']))).show()
projects_df.filter(col('state') == 'successful')\
           .groupBy('main_category', 'category')\
           .count()\
           .withColumn('percentage', col('count')/sum_('count').over(Window.partitionBy(projects_df['main_category'])))\
           .filter(col('main_category') == 'Publishing')\
           .sort(desc('percentage')).show(10)

+-------------+----------------+-----+--------------------+
|main_category|        category|count|          percentage|
+-------------+----------------+-----+--------------------+
|   Publishing|Children's Books| 2160| 0.19213663049279486|
|   Publishing|         Fiction| 2004| 0.17826009606831525|
|   Publishing|      Nonfiction| 2001| 0.17799323963707525|
|   Publishing|      Publishing| 1506| 0.13396192848247643|
|   Publishing|       Art Books| 1250| 0.11119017968333036|
|   Publishing|     Periodicals|  473| 0.04207436399217221|
|   Publishing|          Poetry|  444| 0.03949475182351895|
|   Publishing|Radio & Podcasts|  370| 0.03291229318626579|
|   Publishing|     Anthologies|  206|0.018324141611812846|
|   Publishing|        Academic|  178|0.015833481586906246|
+-------------+----------------+-----+--------------------+
only showing top 10 rows



In [86]:
projects_df.filter(col('state') == 'successful')\
           .groupBy('main_category', 'category')\
           .count()\
           .withColumn('percentage', col('count')/sum_('count').over(Window.partitionBy(projects_df['main_category'])))\
           .withColumn('rank', dense_rank().over(Window.partitionBy('main_category').orderBy(desc('percentage'))))\
           .filter(col('rank') <= 2)\
           .show(30)

+-------------+----------------+-----+-------------------+----+
|main_category|        category|count|         percentage|rank|
+-------------+----------------+-----+-------------------+----+
|         Food|            Food| 2969| 0.5402110625909753|   1|
|         Food|          Drinks|  537|0.09770742358078603|   2|
|          Art|             Art| 3159| 0.2945729205520328|   1|
|          Art|    Illustration| 1535|0.14313688922044013|   2|
|      Fashion|         Fashion| 2031|0.39261550357626135|   1|
|      Fashion|         Apparel| 1308| 0.2528513435144017|   2|
| Film & Video|          Shorts| 6374|0.28409698698520236|   1|
| Film & Video|     Documentary| 5557| 0.2476822963095026|   2|
|   Publishing|Children's Books| 2160|0.19213663049279486|   1|
|   Publishing|         Fiction| 2004|0.17826009606831525|   2|
|       Crafts|          Crafts| 1050| 0.5497382198952879|   1|
|       Crafts|     Woodworking|  242|0.12670157068062826|   2|
|       Comics|          Comics| 2365|0.